# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hamilton,39.1834,-84.5333,33.11,70,0,4.63,US,1690574557
1,1,tromso,69.6496,18.9570,11.88,82,20,4.63,NO,1690575336
2,2,along,28.1667,94.7667,23.36,99,81,0.37,IN,1690575336
3,3,letlhakane,-21.4167,25.5833,17.62,34,19,2.03,BW,1690575337
4,4,san pedro,14.3500,121.0167,24.62,97,75,4.63,PH,1690575028


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
fh = 500
fw = 700
size = 'Humidity'
colors = 'City'
xlab = 'x'
ylab = 'y'

# Display the map
city_data_df.hvplot.points(x='Lng',y='Lat',geo = True,xlabel=xlab,ylabel = ylab,c= colors,s=size,frame_height = fh,frame_width=fw,alpha = 0.5,tiles='OSM')


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria: cloudiness < 20, 21 <= max temperature <= 27, 30 < humidity

filt_city_df = city_data_df[(city_data_df['Cloudiness'] < 20) & (city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <= 27) & (city_data_df['Humidity'] > 30)& (city_data_df['Humidity'] < 70)]
# Drop any rows with null values
filt_city_df = filt_city_df.dropna()

# Display sample data
filt_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,taroudant,30.4703,-8.8770,25.85,51,0,4.17,MA,1690575355
230,230,great falls,47.5002,-111.3008,24.67,41,0,2.57,US,1690575270
242,242,monkey bay,-14.0728,34.9201,22.84,62,2,1.89,MW,1690575399
252,252,ajaccio,42.0000,8.9167,21.03,64,0,1.54,FR,1690575402
273,273,turkmenbasy,40.0222,52.9552,26.43,38,0,3.09,TM,1690575407
292,292,yinchuan,38.4681,106.2731,22.32,64,0,1.38,CN,1690575411
304,304,smara,26.7384,-11.6719,24.92,51,0,6.67,EH,1690575414
323,323,beruniy,41.6911,60.7525,24.98,44,0,4.12,UZ,1690575418
352,352,ilabaya,-17.4208,-70.5133,22.90,36,0,3.88,PE,1690575427
415,415,adamovka,51.5223,59.9396,22.35,62,1,4.04,RU,1690575444


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filt_city_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
79,taroudant,MA,30.4703,-8.8770,51,
230,great falls,US,47.5002,-111.3008,41,
242,monkey bay,MW,-14.0728,34.9201,62,
252,ajaccio,FR,42.0000,8.9167,64,
273,turkmenbasy,TM,40.0222,52.9552,38,
292,yinchuan,CN,38.4681,106.2731,64,
304,smara,EH,26.7384,-11.6719,51,
323,beruniy,UZ,41.6911,60.7525,44,
352,ilabaya,PE,-17.4208,-70.5133,36,
415,adamovka,RU,51.5223,59.9396,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'place': 'categories=accommodation.hotel',
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat,long = row['Lat'],row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'filter=circle:{long},{lat},{radius}'
    params["bias"] = f'bias=proximity:{long},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # name_address = requests.get(f'{base_url}?{params["place"]}&{params["filter"]}&{params["bias"]}&api_key={geoapify_key}')
    name_address = requests.get(f'{base_url}?{"&".join(params.values())}&api_key={geoapify_key}')
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taroudant - nearest hotel: Hotel Tiouit
great falls - nearest hotel: Hotel Arvon
monkey bay - nearest hotel: Chimanga Resthouse
ajaccio - nearest hotel: Hôtel Sole e Monte
turkmenbasy - nearest hotel: Türkmenbaşy Oteli
yinchuan - nearest hotel: Yinchuan hotel
smara - nearest hotel: Hôtel Amine فندق الأمين
beruniy - nearest hotel: Miymanxana
ilabaya - nearest hotel: No hotel found
adamovka - nearest hotel: No hotel found
changji - nearest hotel: No hotel found
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
nelsonville - nearest hotel: No hotel found
balqash - nearest hotel: Престиж
selendi - nearest hotel: No hotel found
fort mcmurray - nearest hotel: Nomad Hotel and Suites
ezine - nearest hotel: No hotel found
sidi bel abbes - nearest hotel: hotel palace
north bend - nearest hotel: The Mill Casino Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
79,taroudant,MA,30.4703,-8.8770,51,Hotel Tiouit
230,great falls,US,47.5002,-111.3008,41,Hotel Arvon
242,monkey bay,MW,-14.0728,34.9201,62,Chimanga Resthouse
252,ajaccio,FR,42.0000,8.9167,64,Hôtel Sole e Monte
273,turkmenbasy,TM,40.0222,52.9552,38,Türkmenbaşy Oteli
292,yinchuan,CN,38.4681,106.2731,64,Yinchuan hotel
304,smara,EH,26.7384,-11.6719,51,Hôtel Amine فندق الأمين
323,beruniy,UZ,41.6911,60.7525,44,Miymanxana
352,ilabaya,PE,-17.4208,-70.5133,36,No hotel found
415,adamovka,RU,51.5223,59.9396,62,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display
# Configure the map plot
fh = 500
fw = 700
size = 'Humidity'
colors = 'City'
xlab = 'x'
ylab = 'y'
cols = list(hotel_df.columns)

# Display the map
hotel_df.hvplot.points(x='Lng',y='Lat',geo = True,xlabel=xlab,ylabel = ylab,c= colors,s=size,frame_height = fh,frame_width=fw,alpha = 0.5,tiles='OSM',hover_cols = cols)



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)